In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10867,2024-02-22,Eua Ncaa,21:00,Eastern Kentucky,North Alabama,1.37,3.29,154.5,1.91,1.91,0.0,0.0,0.00,KICfTzKP,0.729927,0.303951,0.523560,0.523560,0.033878,122.624,27.050598,0.220598,2.4,1.341641,0.559017,102.75,1.478,0.326451,0.220873,33.0,151.964,53.779676,0.353897,1.8,1.643168,0.912871,117.71,2.060,0.609795,0.296017,-1.0,75,79,1.37,1.49,101.086,228.438,0.582680,0.000000,NaN,0.97,0.194,1.907216,0.742425,0.8,0.057575,1.37,0.274,8.357664,0.347777,0.3,-0.047777
10868,2024-02-22,Eua Ncaa,21:00,Northern Kentucky,Cleveland State,1.73,2.18,146.5,1.80,1.95,-3.5,2.0,2.75,lEWUyOID,0.578035,0.458716,0.555556,0.512821,0.036750,128.314,47.097646,0.367050,1.8,1.643168,0.912871,121.80,1.910,0.673535,0.352636,-12.0,145.194,38.727576,0.266730,1.8,1.643168,0.912871,180.63,1.948,0.486642,0.249816,8.0,58,81,2.10,2.23,127.130,152.158,0.162761,0.056569,0.223297,1.29,0.258,2.829457,0.680371,0.8,0.119629,-1.46,-0.292,-4.041096,0.424659,0.4,-0.024659
10869,2024-02-22,Eua Ncaa,21:00,Northeastern,Elon,1.25,4.19,144.5,1.80,1.95,0.0,0.0,0.00,ATjlVhM6,0.800000,0.238663,0.555556,0.512821,0.038663,156.176,33.627236,0.215316,1.2,1.643168,1.369306,200.88,2.270,0.648498,0.285682,-6.0,352.240,263.201343,0.747222,1.8,1.643168,0.912871,766.50,5.314,3.815197,0.717952,-29.0,62,73,3.24,10.50,125.026,328.562,0.764299,0.056569,NaN,-1.64,-0.328,-0.762195,0.565710,0.5,-0.065710,9.13,1.826,1.746988,0.273137,0.3,0.026863
10870,2024-02-22,Eua Ncaa,21:00,New Hampshire,NJIT,1.18,5.40,145.5,1.80,1.95,0.0,0.0,0.00,dfGeqtSP,0.847458,0.185185,0.555556,0.512821,0.032643,209.566,131.210764,0.626107,1.2,1.643168,1.369306,219.24,2.956,1.459188,0.493636,-23.0,232.802,87.863833,0.377419,1.2,1.643168,1.369306,122.10,3.674,1.247229,0.339474,-39.0,54,55,4.06,2.22,119.342,373.348,0.906988,0.056569,NaN,-2.05,-0.410,-0.439024,0.000000,0.0,0.000000,-1.92,-0.384,-11.458333,0.189899,0.2,0.010101
10871,2024-02-22,Eua Ncaa,21:00,Merrimack Warriors,St. Francis (PA),1.11,7.50,133.5,1.80,1.95,0.0,0.0,0.00,tGe3SwSi,0.900901,0.133333,0.555556,0.512821,0.034234,91.506,10.239799,0.111903,3.0,0.000000,0.000000,91.30,1.250,0.146116,0.116893,40.0,226.674,139.749406,0.616522,0.6,1.341641,2.236068,152.46,3.664,2.382421,0.650224,-16.0,83,63,1.10,2.42,87.312,226.342,1.049573,0.056569,NaN,-0.05,-0.010,-11.000000,0.725221,0.8,0.074779,-5.00,-1.000,-6.500000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10967,2024-02-22,Eua Ncaa,20:30,Bellarmine,Central Arkansas,1.69,2.18,139.5,1.80,1.91,0.0,0.0,0.00,0CfNOfSt,0.591716,0.458716,0.555556,0.523560,0.050432,233.010,56.221833,0.241285,0.6,1.341641,2.236068,213.85,3.290,0.956060,0.290596,-37.0,235.674,119.730910,0.508036,0.6,1.341641,2.236068,236.64,3.462,1.277975,0.369144,-40.0,65,68,3.29,3.48,160.946,420.088,0.179061,0.041931,NaN,-3.33,-0.666,-1.036036,0.000000,0.0,0.000000,10.40,2.080,0.567308,0.188845,0.4,0.211155
10968,2024-02-22,Eua Ncaa,20:00,Maine,UMBC Retrievers,1.59,2.35,151.5,1.83,1.83,0.0,0.0,0.00,2ygOv0zs,0.628931,0.425532,0.546448,0.546448,0.054463,187.326,31.506812,0.168192,0.6,1.341641,2.236068,180.69,2.892,0.524805,0.18

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10868,21:00,Eua Ncaa,Northern Kentucky,Cleveland State,1.73,Back Home
10939,21:00,Eua Nba,Philadelphia 76ers,New York Knicks,1.89,Back Home
10960,22:00,Eua Ncaa,Minnesota,Ohio State,1.64,Back Home
